In [2]:
import numpy as np
import keras
import cv2
import glob
import functools

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils

from sklearn.model_selection import train_test_split

from PIL import Image as PImage

from os import listdir

from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Mounted at /content/drive


In [0]:
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1)
top1_acc.__name__ = 'top1_acc'


In [0]:
# Load data
data = []
label = []

def load_image(name,index):
    path = "/content/drive/My Drive/Colab Notebooks/dataset/" + name + "/*JPEG"
    for file in glob.glob(path):
        img = cv2.imread(file) 
        img  = cv2.resize(img,(224,224))
        data.append(img)
        label.append(index)
    return data,label

i=0 
dog_name = ["bullmastiff","chowchow","pug","maltese","huskysibir","dachshund","dalmatian","corgi","chihuahua","yorkshire"]
for name in dog_name:
    print ("loading", i)
    load_image(name,i)
    i +=1

loading 0


NameError: ignored

In [0]:
labels = keras.utils.to_categorical(label,10)

In [0]:
# Split test & train data
(trainX, testX, trainY, testY) = train_test_split(np.array(data),np.array(labels), test_size=0.25)

In [0]:
#model = Sequential()
#vgg = keras.applications.vgg16.VGG16(include_top=True, weights="imagenet")
    
#for layer in vgg.layers:
 # layer.trainable = False

#softmax= Dense(10,activation='softmax')(vgg.layer[-2].output)
  
#model.add(vgg.input)  
#model.add(softmax)

#model.summary()

In [5]:
# Use vgg16 model
vgg = keras.applications.vgg16.VGG16(include_top=True, weights="imagenet")

for layer in vgg.layers:
  layer.trainable = False
  
softmax = Dense(10,activation='softmax')(vgg.layers[-2].output)

model = Model(vgg.input,softmax)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
# Compile model & train 
opt = Adam(lr=1e-4, decay=1e-3 / 50)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy",top3_acc, top1_acc])
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=50, batch_size=32)

Instructions for updating:
Use tf.cast instead.
Train on 9107 samples, validate on 3036 samples
Epoch 1/50
9107/9107 [==============================] - 119s 13ms/step - loss: 0.8461 - acc: 0.7437 - top3_acc: 0.8922 - top1_acc: 0.7437 - val_loss: 0.4294 - val_acc: 0.8633 - val_top3_acc: 0.9641 - val_top1_acc: 0.8633
Epoch 2/50
9107/9107 [==============================] - 112s 12ms/step - loss: 0.3365 - acc: 0.8920 - top3_acc: 0.9750 - top1_acc: 0.8920 - val_loss: 0.3708 - val_acc: 0.8877 - val_top3_acc: 0.9723 - val_top1_acc: 0.8877
Epoch 3/50
9107/9107 [==============================] - 114s 12ms/step - loss: 0.2647 - acc: 0.9131 - top3_acc: 0.9834 - top1_acc: 0.9131 - val_loss: 0.3480 - val_acc: 0.8936 - val_top3_acc: 0.9753 - val_top1_acc: 0.8936
Epoch 4/50
9107/9107 [==============================] - 114s 12ms/step - loss: 0.2223 - acc: 0.9271 - top3_acc: 0.9872 - top1_acc: 0.9271 - val_loss: 0.3346 - val_acc: 0.8995 - val_top3_acc: 0.9766 - val_top1_acc: 0.8995
Epoch 5/50
9107/9107